# 智能推荐系统

## 智能推荐系统的基本原理
### 智能推荐系统的基础：协同过滤算法
协同过滤算法的原理是根据用户群体的产品偏好数据，发现用户或物品之间的相关性，并基于这些相关性为用户进行推荐。根据原理的不同，协同过滤算法分为两类——基于用户的协同过滤算法和基于物品的协同过滤算法。

#### 1、基于用户的协同过滤算法
基于用户的协同过滤算法的本质是寻找相似的用户：通过一个用户的相关数据寻找与该用户相似的其他用户，进而为该用户推荐相似用户关注的产品。

如下表所示，用户1和用户2都给商品A、B、C打了高分，那么可以将用户1和用户2划分在同一个用户群体，此时若用户2还给商品D打了高分，那么就可以将商品D推荐给用户1。
![基于用户的协同过滤](image/基于用户的协同过滤.png)

#### 2、基于物品的协同过滤算法
基于物品的协同过滤算法的本质是寻找相似的物品：通过一个物品的相关数据寻找与该物品相似的其他物品，进而为关注该物品的用户推荐相似的物品。

如下表所示，图书A和图书B都被用户1、2、3购买过（1表示购买，0表示未购买），那么可以认为图书A和图书B具有较强的相似度，进而推测喜欢图书A的用户同样也会喜欢图书B。当用户4购买图书B时，根据图书A和图书B的相似性，可将图书A推荐给用户4。
![基于物品的协同过滤](image/基于物品的协同过滤.png)

**在商业实战中，大多数应用场景偏向于使用基于物品的协同过滤算法**，主要原因如下：
* 通常情况下，用户的数量是非常庞大的（如淘宝的用户群体达数亿），而物品的数量则相对有限，因此，计算不同物品的相似度往往比计算不同用户的相似度容易很多。
* 用户的喜好较为多变，而物品的属性较明确，不随时间变化，过去的用户对物品的评分长期有效，所以物品的相似度比较固定。可以预先离线计算好物品的相似度，把结果存在表中，需要向用户进行推荐时再从表中调用。

## 计算相似度的常用方法
无论是基于用户还是基于产品的协同过滤算法，其本质都是寻找数据之间的相似度。计算相似度的常用方法有——**欧氏距离、余弦值和皮尔逊相关系数**。

下表为3个用户对3种物品的评分，5表示五星评级，依次类推。将使用该表讲解计算相似度的3种方法的基本原理。
![计算相似度表例.png](image/计算相似度表例.png)
> 表中的评分数据都在0～5之间，量级一致，所以无须做标准化处理。如果数据的量级存在较大差异，应先做标准化处理

### （1）欧氏距离

欧氏距离引入三维向量空间下的欧氏距离来刻画两点间的距离。在空间几何中，如果A点坐标为（X1，X2，X3），B点坐标为（Y1，Y2，Y3），则A点与B点间的欧氏距离计算公式如下。

$d(A, B) = \sqrt{(X1-Y1)^2 + (X2-Y2)^2 + (X3-Y3)^2}$ 

该公式同样可以衡量物品A和物品B之间的距离。d（A，B）代表物品A和物品B间的欧氏距离，X1、X2、X3分别代表用户1、用户2、用户3对物品A的评分，Y1、Y2、Y3分别代表用户1、用户2、用户3对物品B的评分。d（A，B）越小，说明两者越接近，则两者越相似。将前面的用户评分表中的数据代入公式，计算物品A和物品B的欧氏距离为：

$d(A, B) = \sqrt{(5-4)^2 + (1-2)^2 + (5-2)^2} = \sqrt{11} = 3.32$

除了直接比较欧氏距离，还可以利用欧氏距离衍生出的相似度公式来衡量两者的相似度。基于欧氏距离的相似度sim（A，B）定义如下。由公式可知，两物品间的欧氏距离越小，则相似度越大，两物品越相近。

$sim（A，B） = \frac x{1 + d(A,B)}$

将前面计算出的欧氏距离代入公式，计算物品A和物品B的相似度为：

$sim（A，B） = \frac 1{1 + 3.32} = 0.23$

![欧氏距离.png](image/欧氏距离.png)

前面的用户评分表分别计算欧氏距离和相似度，物品B和C的欧氏距离最小，相似度最大，可以认为在这3种物品中，物品B和C最为相似，喜欢物品B的用户有较大的可能也喜欢物品C，因此可以向购买了物品B的用户推荐物品C，向购买了物品C的用户推荐物品B。

In [2]:
# 通过NumPy库可以方便地计算两个向量的欧氏距离
import pandas as pd
df = pd.DataFrame([[5, 1, 5], [4, 2, 2], [4, 2, 1]],  columns=['用户1', '用户2', '用户3'], index=['物品A', '物品B', '物品C'])

# 计算物品A和物品B的欧氏距离
import numpy as np
dist = np.linalg.norm(df.iloc[0] - df.iloc[1])  # df.iloc[0]表示DataFrame的第1行数据，即物品A的数据，df.iloc[1]则表示第2行数据，即物品B的数据
dist

3.3166247903554

### （2）余弦相似度
在n维向量空间中，若向量a＝（X1，X2，X3，…，Xn），向量b＝（Y1，Y2，Y3，…，Yn），则向量a和向量b的夹角的余弦值可以表示为：
![余弦相似度.png](image/余弦相似度.png)
根据前面的用户评分表，物品A对应的向量a为（5，1，5），物品B对应的向量b为（4，2，2），代入上述公式可得：
![余弦相似度计算.png](image/余弦相似度计算.png)
向量a和向量b的余弦相似度为0.91，说明物品A和物品B十分相似。

In [4]:
# 利用Scikit-Learn库中的cosine_similarity()函数计算各物品间的余弦相似度，并整理成DataFrame格式的二维表格
from sklearn.metrics.pairwise import cosine_similarity
user_similarity = cosine_similarity(df)
pd.DataFrame(user_similarity,  columns=['物品A', '物品B', '物品C'], index=['物品A', '物品B', '物品C'])

,物品A,物品B,物品C
物品A,1.000000,0.914659,0.825029
物品B,0.914659,1.000000,0.979958
物品C,0.825029,0.979958,1.000000


### （3）皮尔逊相关系数
皮尔逊相关系数r是用于描述两个变量间线性相关强弱程度的统计量，取值范围为[-1，1]，为正值代表两个变量存在正相关，为负值代表两个变量存在负相关。r的绝对值越大，说明相关性越强。
使用皮尔逊相关系数度量物品间的相似度时，皮尔逊相关系数的绝对值越大，两个物品的线性相关程度越强，两者越相似。
**对于实战来说，不需要深究皮尔逊相关系数的数学推导，因为有许多现成的计算库可以使用。**

In [5]:
from scipy.stats import pearsonr
x = [1, 3, 5, 7, 9]
y = [9, 8, 6, 4, 2]
corr = pearsonr(x, y)
print('相关系数r值为' + str(corr[0]) + '，显著性水平P值为' + str(corr[1]))

相关系数r值为-0.9938837346736191，显著性水平P值为0.0005736731093321747


只需给pearsonr()函数传入两个数组，它就能返回一个元组，其中包含两个浮点数：皮尔逊相关系数r值和显著水平P值。P值与显著性检验有关，P＜0.05表示显著相关，即两个变量之间的相关性真的存在，而不是由偶然因素引起的。只有在显著相关的前提下，r值才有意义。简单来说就是先通过观察P值是否小于0.05判定是否真的存在相关性，再根据r值判定相关性有多强。上述代码的运行结果如下。可以看到，r值约为-0.99，可以初步认为X和Y具有很强的负相关性，且P值约为0.00057，满足小于0.05的条件，说明X和Y的确具有相关性。

pandas库中DataFrame提供的corrwith()函数可以计算单个物品与其他物品的皮尔逊相关系数，corr()函数可以计算整张表的皮尔逊相关系数。因为这两个函数默认计算的是DataFrame的列与列之间的相关系数，所以需要先将上述构造的用户评分表df进行转置，用行代表用户，用列代表物品。

In [6]:
df = df.T
df.head()

,物品A,物品B,物品C
用户1,5,4,4
用户2,1,2,2
用户3,5,2,1


In [8]:
# 用corrwith()函数计算物品A与其他物品的皮尔逊相关系数
A = df['物品A']
corr_A = df.corrwith(A)
corr_A

物品A    1.000000
物品B    0.500000
物品C    0.188982
dtype: float64

可以看到，物品A与本身的r值为1，与物品B的r值为0.5，与物品C的r值约为0.1890。

In [11]:
# 计算表中各物品间的皮尔逊相关系数
df.corr()

,物品A,物品B,物品C
物品A,1.000000,0.500000,0.188982
物品B,0.500000,1.000000,0.944911
物品C,0.188982,0.944911,1.000000


可以看出，物品B和C的r值的绝对值最大，约为0.9449，因此可以认为表中的所有物品中它们最相似，这与使用欧氏距离和余弦相似度度量物品间相似度所得出的结论完全一致。